In [ ]:
v
-- CREATE SCHEMA IF NOT EXISTS gold;

-- -- -- Create Tables

-- CREATE TABLE gold.dim_customers (
--     customer_key      INT,
--     customer_id       INT,
--     customer_number   VARCHAR(50),
--     first_name        VARCHAR(50),
--     last_name         VARCHAR(50),
--     country           VARCHAR(50),
--     marital_status    VARCHAR(50),
--     gender            VARCHAR(50),
--     birthdate         DATE,
--     create_date       DATE
-- );

-- CREATE TABLE gold.dim_products (
--     product_key    INT,
--     product_id     INT,
--     product_number VARCHAR(50),
--     product_name   VARCHAR(50),
--     category_id    VARCHAR(50),
--     category       VARCHAR(50),
--     subcategory    VARCHAR(50),
--     maintenance    VARCHAR(50),
--     cost           INT,
--     product_line   VARCHAR(50),
--     start_date     DATE
-- );

-- CREATE TABLE gold.fact_sales (
--     order_number  VARCHAR(50),
--     product_key   INT,
--     customer_key  INT,
--     order_date    DATE,
--     shipping_date DATE,
--     due_date      DATE,
--     sales_amount  INT,
--     quantity      SMALLINT,
--     price         INT
-- );
-- select* from gold.fact_sales
-- -- Truncate Tables before loading
-- TRUNCATE TABLE gold.dim_customers;
-- TRUNCATE TABLE gold.dim_products;
-- TRUNCATE TABLE gold.fact_sales;

--queries to track change over time
-- sales over years
select     
EXTRACT(YEAR FROM order_date) AS order_year,
sum(sales_amount) as total_sales ,
count(distinct customer_key) as total_customers,
sum(quantity) as total_quantity
from gold.fact_sales
where order_date is not null 
group by order_year
order by order_year;
-- sales over the months
select     
EXTRACT(month FROM order_date) AS order_month,
sum(sales_amount) as total_sales ,
count(distinct customer_key) as total_customers,
sum(quantity) as total_quantity
from gold.fact_sales
where order_date is not null 
group by order_month
order by order_month
-- sales by year and month
select 
EXTRACT(year FROM order_date) AS order_year,
EXTRACT(month FROM order_date) AS order_month,
sum(sales_amount) as total_sales ,
count(distinct customer_key) as total_customers,
sum(quantity) as total_quantity
from gold.fact_sales
where order_date is not null 
group by order_month,order_year
order by order_month,order_year
-- queries for cumulative analysis using window functions

SELECT 
    order_month,
    total_sales,
    SUM(total_sales) OVER (ORDER BY order_month) AS running_total_sales
FROM (
    SELECT 
        DATE_TRUNC('month', order_date) AS order_month,
        SUM(sales_amount) AS total_sales
    FROM gold.fact_sales
    WHERE order_date IS NOT NULL
    GROUP BY DATE_TRUNC('month', order_date)
) AS monthly_sales
ORDER BY order_month;


SELECT 
    order_month,
    total_sales,
    SUM(total_sales) OVER ( partition by order_month ORDER BY order_month) AS running_total_sales
FROM (
    SELECT 
        DATE_TRUNC('month', order_date) AS order_month,
        SUM(sales_amount) AS total_sales
    FROM gold.fact_sales
    WHERE order_date IS NOT NULL
    GROUP BY DATE_TRUNC('month', order_date)
) AS monthly_sales
ORDER BY order_month;
-- note, here when we partition, for each date partition it resets the starting total sales so it accumulates by month not overall

SELECT 
    order_month,
    total_sales,
    SUM(total_sales) OVER ( partition by order_month ORDER BY order_month) AS running_total_sales,
	avg(avg_price) OVER ( partition by order_month ORDER BY order_month) AS moving_average_price
FROM (
    SELECT 
        DATE_TRUNC('month', order_date) AS order_month,
        SUM(sales_amount) AS total_sales,
		avg(price) as avg_price
    FROM gold.fact_sales
    WHERE order_date IS NOT NULL
    GROUP BY DATE_TRUNC('month', order_date)
) AS monthly_sales
ORDER BY order_month;






